In [18]:
# ! pip install psycopg2-binary
# ! pip install sqlparse

In [13]:
# # Establish a connection to the PostgreSQL database
# from langchain_community.utilities import SQLDatabase
# import psycopg2
# password='Jakh=J0hn0103'
# # host='128.179.210.75'
# host='192.168.0.66'
# # host='10.251.0.127'
# db='catastici'

# conn = psycopg2.connect(
#     host=host,
#     port=5432,
#     user='postgres',
#     password=password,
#     database=db
# )

# # Test the connection
# cursor = conn.cursor()
# cursor.execute("SELECT * FROM catastici LIMIT 1")
# rows = cursor.fetchall()
# for row in rows:
#     print(row)
# conn.close()

# # Setup database
# db = SQLDatabase.from_uri(f"postgresql+psycopg2://postgres:{password}@{host}:5432/{db}")

# # test DB
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM catastici LIMIT 1;")

In [25]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///catastici.db")

# test DB
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM catastici LIMIT 1;")

sqlite
['catastici']


"[(0, 3173, 'liberal', 'campi', 'casa e bottega da barbier', 70, 'campo vicino alla chiesa')]"

# Experiment

In [70]:
import pandas as pd
questions = pd.read_csv('test_data.csv')
questions

,question_id,level_len,level_nest,question,true_query,true_answer,matched_contents
0,0,0,0,What are the various kinds of properties ident...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN
1,0,0,0,Could you enumerate the different property cat...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN
2,0,0,0,"In the dataset, what property types can be found?","SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN
3,0,0,0,Specify all the property types that are encaps...,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN
4,0,0,0,List all the types of properties in the dataset.,"SELECT DISTINCT ""Property_Type"" \nFROM catastici;","[('casa e bottega da barbier',), ('casa',), ('...",NaN
...,...,...,...,...,...,...,...
495,99,1,1,How does the rent income fluctuate on average ...,SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN
496,99,1,1,"Across various locations, what variation exist...",SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN
497,99,1,1,What's the range of differences in average ren...,SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN
498,99,1,1,Can you identify the usual disparity in rent i...,SELECT AVG(Variance) AS Avg_Rent_Income_Varian...,"[(6150.230341110285,)]",NaN


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True,
)

/home/saydalie/.local/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [71]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Search for exact match with "=", unless specified otherwise

### Database Schema
CREATE TABLE [catastici]
(
    [Property_ID] INTEGER PRIMARY KEY,
    [Owner_ID] INTEGER, -- Unique ID of each owner of the property
    [Owner_First_Name] VARCHAR(30), -- First name of the owner of the property
    [Owner_Family_Name] VARCHAR(30), -- Family name of the owner of the property
    [Property_Type] VARCHAR(100), -- Specific type of the property given in Italian
    [Rent_Income] INTEGER, -- Rent price of the property that the owner receives as income, given in Venice ancient gold coin ducats
    [Property_Location] VARCHAR(100) -- Ancient approximate toponym of the property given in Italian
);

### Matched contents
{matched_contents}

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [72]:
import sqlparse

def generate_query(query):
    updated_prompt = prompt.format(question=query['question'], matched_contents=query['matched_contents'])
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=4,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [75]:
from tqdm.notebook import tqdm
tqdm.pandas()

generated_query = questions.progress_apply(generate_query, axis=1)
questions["generated_query"] = generated_query

  0%|          | 0/500 [00:00<?, ?it/s]

In [76]:
questions.to_csv('final.csv',index=False)

# Evaluations

In [87]:
import re

def check_sql_executability(query):
    try:
        return db.run(query)
    except Exception as e:
        return str(e)

def ngram_overlap(true_answer, generated_answer):
    """Calculates 1-gram overlap"""
    # Convert lists to sets to remove duplicates
    true_set = set(true_answer)
    generated_set = set(generated_answer)
    
    # Calculate the intersection of the sets
    overlap_count = len(true_set.intersection(generated_set))
    
    # Calculate the percentage of overlap
    overlap_percentage = overlap_count / len(true_set) if len(true_set) > 0 else 0.0
    
    return round(overlap_percentage, 3)

def clean_answer(answer, to_replace = ['[', ']', '(', ',)', "'", ')']):
    pattern = '|'.join(map(re.escape, to_replace))
    cleaned_answer = re.sub(pattern, '', answer)
    return [ans.strip() for ans in cleaned_answer.split(',')]

In [88]:
questions['generated_answer'] = questions['generated_query'].apply(check_sql_executability)

In [92]:
questions['true_answer'].fillna('', inplace=True)

In [93]:
questions['generated_answer_clean'] = questions['generated_answer'].apply(clean_answer)
questions['true_answer_clean'] = questions['true_answer'].apply(clean_answer)
questions['n_gram'] = questions.apply(lambda row: ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']), axis=1)

In [94]:
questions['output'] = questions['n_gram']>0.33
questions['output'].value_counts()

output
True     295
False    205
Name: count, dtype: int64